In [1]:
from functions import *
import argparse
from astropy.io import fits
from astropy.wcs import WCS
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.patches import Ellipse
import warnings
warnings.filterwarnings('ignore')
import os
from os import path

rootdirectory='/Volumes/TARDIS/Work/askap/'
defaultfile=rootdirectory+'possum_plots_sourcelist.txt'
fig_directory='/Users/emma/OneDrive/PhD/thesis/Figures/'

#------------------------file paths------------------------

#set up paths to load and save files 
rootdirectory='/Volumes/TARDIS/Work/askap/'
fig_directory='/Users/emma/OneDrive/PhD/thesis/Figures/'
#Galacitc RM map
galacicRMfile='/Volumes/TARDIS/Work/askap/Faraday_cutout_Oph.fits'
galacticRMerrs='/Volumes/TARDIS/Work/askap/Faraday_error_Oph.fits'

src='1705-2445' #this is used when naming plots saved to file
directory=rootdirectory+src+'/' #I have sub folders for each source

#------------------------plot parameters------------------------

#user settings
rm_scale=40 # cutoff for colour scale on RM plots
xmin,xmax=55,195 #where to display data
ymin,ymax=40,140 
ithresh=3*0.00078042 #total intensity threshold for contour plots (typically 3x local rms noise)
savefigs=False #save figures to file?

sig_crop_pix=20

#set parameters for plots
plasmacmap=plt.cm.plasma
plasmacmap.set_bad('grey',1)
wisecmap=plt.cm.Greys
wisecmap.set_bad('white',1)
rmcmap=plt.cm.seismic
rmcmap.set_bad('gray',0.5)
magmacmap=plt.cm.magma
magmacmap.set_bad('gray',0.5)

twilightcmap=plt.cm.twilight

plt.rcParams.update({'lines.linewidth':0.3})

plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'cmu serif'

SMALL_SIZE = 7
MEDIUM_SIZE = 8
BIGGER_SIZE = 9

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize

#read in fits files
# total intensity image and image cubes
#fitsopen is from functions.py and is a condensed use of astropy.fits.open
cont_im,header=fitsopen(directory+src+'_cont.fits')
i_cube_data,cube_head=fitsopen(directory+src+'_i.fits')
q_cube_data,qcube_head=fitsopen(directory+src+'_q.fits')
u_cube_data,ucube_head=fitsopen(directory+src+'_u.fits')

#files from other codes
#Peak rm from seperate FDF fitting code
pkrm_fit,pkrm_fit_header=fitsopen(directory+'FDF_peakRM_fitted.fits')
chi0_fit,chi0_fit_header=fitsopen(directory+'FDF_chi0_fitted.fits')
fdf_width,fdfwidthhead=fitsopen(directory+'FDF_peakRM_fitted_width.fits')

#FDF files from RM-tools rm synthesis
#fdf
fdfdata,fdfheader=fitsopen(directory+'FDF_tot_dirty.fits')
fdf_im,fdfhead=fitsopen(directory+'FDF_im_dirty.fits')
fdf_real,fdfhead=fitsopen(directory+'FDF_real_dirty.fits')
fdf_clean,fdf_clean_head=fitsopen(directory+'FDF_clean_tot.fits')
clean_components,cc_head=fitsopen(directory+'FDF_CC_tot.fits')
rmsf_FHWMarr,rmsfhead=fitsopen(directory+'RMSF_FWHM.fits')
rmsf_FWHM=np.nanmean(rmsf_FHWMarr)

#frequencies and wavelengths
ref_freq=cube_head['CRVAL3']
ref_freq_pos=cube_head['CRPIX3']
delta_freq=cube_head['CDELT3']
nfreqchans=cube_head['NAXIS3']
freqs=np.divide(np.linspace(ref_freq,(ref_freq+(delta_freq*nfreqchans)),nfreqchans),1000000000)
c=299792458 #m/s
lamda=np.divide(c,(freqs*1000000000))
lamdasq=np.square(lamda)

#faraday depth space info
fdf_ref_rm=fdfheader['CRVAL3']
fdf_ref_chan=fdfheader['CRPIX3']
fdfres=fdfheader['CDELT3']
nrmchans=fdfheader['NAXIS3']
fdfmin=fdf_ref_rm-(fdf_ref_chan*fdfres) + 1
rmarray=np.arange(fdfmin,fdfmin+(fdfres*nrmchans),fdfres)